In [1]:
import numpy as np
import qpms
import math
from qpms.qpms_p import cart2sph, sph2cart, sph_loccart2cart, sph_loccart_basis
from qpms.cybspec import BaseSpec
from qpms.cytmatrices import CTMatrix, TMatrixGenerator
from qpms.qpms_c import Particle
from qpms.cymaterials import EpsMu, EpsMuGenerator, LorentzDrudeModel, lorentz_drude
from qpms.cycommon import DebugFlags, dbgmsg_enable
from qpms import FinitePointGroup, ScatteringSystem, BesselType, eV, hbar
from qpms.symmetries import point_group_info
eh = eV/hbar
pi = math.pi
def cleanarray(a, atol=1e-10, copy=True):
    a = np.array(a, copy=copy)
    sieve = abs(a.real) < atol
    a[sieve] = 1j * a[sieve].imag
    sieve = abs(a.imag) < atol
    a[sieve] = a[sieve].real
    return a
def nocleanarray(a):
    return a



In [2]:
Nx, Ny = 3, 3
px, py = 580e-9, 580e-9
orig_x = (np.arange(Nx/2) + (0 if (Nx % 2) else .5)) * px
orig_y = (np.arange(Ny/2) + (0 if (Ny % 2) else .5)) * py

orig_xy = np.stack(np.meshgrid(orig_x, orig_y), axis = -1)

omega=1.8*eh
bspec = BaseSpec(lMax=1)
metal = EpsMuGenerator(lorentz_drude['Au'])
medium = EpsMuGenerator(EpsMu(1.52**2))
tmatrix = TMatrixGenerator.cylinder(outside=medium, inside=metal, r=50e-9, h=50e-9)
particles= [Particle(orig_xy[i], tmatrix, bspec=bspec) for i in np.ndindex(orig_xy.shape[:-1])]

sym = FinitePointGroup(point_group_info['D2h'])
ss, ssw = ScatteringSystem.create(particles, medium, omega, sym=sym)
wavenumber = medium.k(omega).real

In [3]:
theta = np.array((-1,.1,-1e-7,0,1e-7,.1,1))#np.linspace(-1,1,5)
phi = 0#pi/2
psi = 0#pi/2
chi = 0#-pi/4

k_sph_list = np.stack((np.broadcast_to(wavenumber, theta.shape), theta, np.broadcast_to(phi, theta.shape)), axis=-1)
sψ, cψ = math.sin(psi), math.cos(psi)
sχ, cχ = math.sin(chi), math.cos(chi)
E_sph = cleanarray((0., cψ*cχ + 1j*sψ*sχ, sψ*cχ + 1j*cψ*sχ) )
k_cart_list = cleanarray(sph2cart(k_sph_list))
E_cart_list = cleanarray(sph_loccart2cart(E_sph, k_sph_list))

npoints = theta.shape[0]



/home/mmn/.local/lib/python3.7/site-packages/ipykernel_launcher.py:17: ComplexWarning: Casting complex values to real discards the imaginary part


In [4]:
E_sph

array([0.+0.j, 1.+0.j, 0.+0.j])

In [5]:
k_sph_list

array([[ 1.38653114e+07, -1.00000000e+00,  0.00000000e+00],
       [ 1.38653114e+07,  1.00000000e-01,  0.00000000e+00],
       [ 1.38653114e+07, -1.00000000e-07,  0.00000000e+00],
       [ 1.38653114e+07,  0.00000000e+00,  0.00000000e+00],
       [ 1.38653114e+07,  1.00000000e-07,  0.00000000e+00],
       [ 1.38653114e+07,  1.00000000e-01,  0.00000000e+00],
       [ 1.38653114e+07,  1.00000000e+00,  0.00000000e+00]])

In [6]:
E_cart_list

array([[ 5.40302306e-01+0.j,  0.00000000e+00+0.j,  8.41470985e-01+0.j],
       [ 9.95004165e-01+0.j,  0.00000000e+00+0.j, -9.98334166e-02+0.j],
       [ 1.00000000e+00+0.j,  0.00000000e+00+0.j,  1.00000000e-07+0.j],
       [ 1.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j],
       [ 1.00000000e+00+0.j,  0.00000000e+00+0.j, -1.00000000e-07+0.j],
       [ 9.95004165e-01+0.j,  0.00000000e+00+0.j, -9.98334166e-02+0.j],
       [ 5.40302306e-01+0.j,  0.00000000e+00+0.j, -8.41470985e-01+0.j]])

In [7]:
np.linalg.norm(E_cart_list, axis=-1)

array([1., 1., 1., 1., 1., 1., 1.])

In [8]:
j = 3
a0 = cleanarray(ss.planewave_full(k_cart=k_cart_list[j], E_cart=E_cart_list[j]))
k_cart_list[j], E_cart_list[j]

(array([       0.        ,        0.        , 13865311.35534979]),
 array([1.+0.j, 0.+0.j, 0.+0.j]))

In [9]:
a0.reshape((ss.positions.shape[0],len(bspec)))[0,:]

array([ 3.06998012+0.j,  0.        +0.j, -3.06998012+0.j, -3.06998012+0.j,  0.        +0.j, -3.06998012+0.j])

In [10]:
ss.positions

array([[ 0.0e+00,  0.0e+00,  0.0e+00],
       [ 5.8e-07,  0.0e+00,  0.0e+00],
       [ 0.0e+00,  5.8e-07,  0.0e+00],
       [ 5.8e-07,  5.8e-07,  0.0e+00],
       [-5.8e-07, -0.0e+00, -0.0e+00],
       [ 0.0e+00, -5.8e-07,  0.0e+00],
       [-5.8e-07,  5.8e-07, -0.0e+00],
       [-5.8e-07, -5.8e-07,  0.0e+00],
       [ 5.8e-07, -5.8e-07, -0.0e+00]])

In [11]:
wavenumber

13865311.355349794

In [12]:
bspec.tlm()

(array([2, 2, 2, 1, 1, 1], dtype=int32),
 array([1, 1, 1, 1, 1, 1], dtype=int32),
 array([-1,  0,  1, -1,  0,  1], dtype=int32))

In [13]:
a0.shape

(54,)

In [14]:
import os
os.getpid()

4274